---
title: Downloading Climate Data from ERA5-Land
short_title: ERA5 Climate Data
---

In this notebook we will demonstrate how we can use Climate Tools and `dhis2eo` to retrieve climate data for a set of DHIS2 organisation units, based on the [ERA5-Land dataset](https://cds.climate.copernicus.eu/datasets/derived-era5-single-levels-daily-statistics?tab=download), hosted at the [ECMWF Climate Data Store (CDS)](https://cds.climate.copernicus.eu/datasets). 

----------------------------------------
## Requirements

### 1. Connect to DHIS2

In order to run this notebook, you first need to connect to an instance of DHIS2. For our example, we will connect to a local instance of DHIS2 containing the standard Sierra Leone demo database, but you should be able to switch out the instance url and credentials to work directly with your own database. 

In [ ]:
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Create DHIS2 client connection
cfg = ClientSettings(
  base_url="https://play.im.dhis2.org/stable-2-42-3",#"http://localhost:8080",
  username="admin",
  password="district"
)
client = DHIS2Client(settings=cfg)

# Verify connection
info = client.get_system_info()
print("Current DHIS2 version:", info["version"])

Current DHIS2 version: 2.42.3


### 2. Register for ERA5 Data Access

#### Authenticate with your ECMWF user

Before you can download the dataset programmatically, you need to [create an ECMWF user](https://www.ecmwf.int/user/login), and authenticate using your user credentials:

- Go to the [CDSAPI Setup page](https://cds.climate.copernicus.eu/how-to-api) and make sure to login.
- Once logged in, scroll down to the section "Setup the CDS API personal access token". 
  - This should show your login credentials, and look something like this:

        url: https://cds.climate.copernicus.eu/api
        key: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

- Copy those two lines to a file `.cdsapirc` in your user's $HOME directory.

#### Accept the dataset license

ECMWF requires that you manually accept the user license for each dataset that you download. 

- Start by visiting the Download page of the dataset we are interested in: ["ERA5 post-processed daily statistics on single levels from 1940 to present"](https://cds.climate.copernicus.eu/datasets/derived-era5-single-levels-daily-statistics?tab=download). 
- Scroll down until you get to the "Terms of Use" section.
- Click the button to accept and login with your user if you haven't already. 

--------------------------------------------
## Step-by-step Workflow

We start by importing the necessary libraries:

In [ ]:
import dhis2eo
import dhis2eo.org_units
import dhis2eo.data.cds

### Step 1: Retrieve organisation units

Before we can download the data, we need to load our organisation units in order to limit which region to download data for.

First we retrieve the organisation units as a GeoJSON dict from the `dhis2-python-client`: 

In [5]:
org_unit_level = 2
org_units_geojson = client.get_org_units_geojson(level=org_unit_level)

Next, load this GeoJSON dict as a `geopandas.GeoDataFrame` by using the `dhis2eo.org_units` module. This makes it easier work with the organisation units for later steps: 

In [6]:
org_units = dhis2eo.org_units.from_dhis2_geojson(org_units_geojson)
org_units

,org_unit_id,name,level,geometry
0,O6uvpzGd5pu,Bo,2,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,fdc6uOvgoji,Bombali,2,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,lc3eMKXaEfw,Bonthe,2,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,jUb8gELQApl,Kailahun,2,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,PMa2VCrupOd,Kambia,2,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,kJq2mPyFEHo,Kenema,2,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,qhqAxPSTUXp,Koinadugu,2,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Vth0fbpFcsO,Kono,2,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,jmIPBj66vD6,Moyamba,2,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,TEQlaapDQoK,Port Loko,2,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


### Step 2: Download ERA5 data for a single month

In order to get users started, we provide a convenience function for downloading the most commonly requested climate variables from the [ERA5 post-processed daily statistics on single levels from 1940 to present](https://cds.climate.copernicus.eu/datasets/derived-era5-single-levels-daily-statistics). 

Simply provide the year, month, and org_units you want to download for. The region to download data for is automatically calculated from the provided organisation units:

In [ ]:
data = dhis2eo.data.cds.get_daily_era5_data(2021, 1, org_units)
data.to_xarray()

dhis2eo.data.cds - INFO - Downloading daily_mean data from CDS API...
dhis2eo.data.cds - INFO - Request parameters: 
{"product_type": "reanalysis", "variable": ["2m_temperature"], "year": "2021", "month": ["01"], "daily_statistic": "daily_mean", "time_zone": "utc+00:00", "frequency": "1_hourly", "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived", "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"]}


2025-11-10 21:35:43,068 INFO Request ID is c42d950e-0254-445b-bc31-87ff473ca7ae
2025-11-10 21:35:43,131 INFO status has been updated to accepted
2025-11-10 21:35:56,630 INFO status has been updated to running
2025-11-10 21:36:58,643 INFO status has been updated to successful


230d486477d28a8421d7dc153c0871c7.nc:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

dhis2eo.data.cds - INFO - Downloading daily_sum data from CDS API...
dhis2eo.data.cds - INFO - Request parameters: 
{"product_type": "reanalysis", "variable": ["total_precipitation"], "year": "2021", "month": ["01"], "daily_statistic": "daily_sum", "time_zone": "utc+00:00", "frequency": "1_hourly", "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived", "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"]}


2025-11-10 21:37:03,713 INFO Request ID is a1677c53-bc18-4fab-8286-aa9feab34416
2025-11-10 21:37:03,770 INFO status has been updated to accepted
2025-11-10 21:37:17,225 INFO status has been updated to running
2025-11-10 21:38:57,647 INFO status has been updated to successful


f97ee70a30be4236361f61b83aeecd72.nc:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

dhis2eo.data.cds - INFO - Saving to cache: /tmp/cds_daily-era5_params-ca5bab_region-37098a_2021-01.nc


/usr/local/python/3.12.1/lib/python3.12/site-packages/earthkit/data/readers/netcdf/fieldlist.py:202: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  return xr.open_mfdataset(


<xarray.Dataset> Size: 41kB
Dimensions:     (valid_time: 30, latitude: 13, longitude: 13)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 240B 2021-01-01 ... 2021-01-30
  * latitude    (latitude) float64 104B 9.918 9.668 9.418 ... 7.418 7.168 6.917
  * longitude   (longitude) float64 104B -13.3 -13.05 -12.8 ... -10.55 -10.3
    number      int64 8B ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 20kB dask.array<chunksize=(30, 13, 13), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 20kB dask.array<chunksize=(30, 13, 13), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-10T21:36 GRIB to CDM+CF via cfgrib-0.9.1...

Note: Since data downloads can be slow, this function also caches the download results and reuses it if the file has already been downloaded. 

### Step 3: Downloading data for multiple months

For most practical examples we want to download data for all months in a time period, which we can do with a simple loop. For the sake of the notebook, we only loop through the periods and don't actually run the download functions which would be too slow.

In [ ]:
start_year = 2022
start_month = 3
end_year = 2025
end_month = 9

for year in range(start_year, end_year+1):
    for month in range(1, 12+1):
        
        # skip months before or after our defined time range
        if (year,month) < (start_year,start_month):
            continue
        if (year,month) > (end_year,end_month):
            continue

        # download the climate data
        # commented out for this notebook
        print(f'Should download data for {year}-{month}')
        #data = dhis2eo.data.cds.get_daily_era5_data(2021, 1, org_units)

        # do something with the data
        # e.g. save to disk, aggregate, or import to DHIS2
        # ...

## Next steps

In this notebook we have shown how to use `dhis2eo` for automatically fetching ERA5 climate data for defined time period and a set of DHIS2 organisation units. 

But there are still several steps that remains to be performed to do something useful with these data, such as: 

* [Aggregate the data to your organisation units](../../aggregation/org-unit-aggregation.ipynb).
* [Import the aggregated data to DHIS2](../../import-data/using-python-client.ipynb).
* Automatically detect if data has already been imported (guide to be added soon).
* Automatically trigger this script at regular intervals, e.g. via a `cron` job (guide to be added in the future). 